In [1]:
# dependencies
from sqlalchemy import create_engine, inspect, MetaData, select, text, Table, func
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Create an engine that can talk to the database
engine = create_engine("sqlite:///Resources/la_crime.db", connect_args={'timeout': 30}, echo=False)
conn = engine.connect()

## Checking Database Data - using SQL

In [2]:
# Query data from crime table
crime_data = conn.execute("SELECT * FROM crime")

i = 0
for record in crime_data:
    print(record)
    i += 1
    if i == 10:
        break

(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501)
(20305364, 2019, 1, 'January', '41-50', 'Hispanic', 'Male', 34.0055, -118.2915, 3, 626, 502)
(170701073, 2018, 1, 'January', '11-20', 'Black', 'Male', 34.0421, -118.3456, 7, 888, 501)
(180100001, 2018, 9, 'September', '31-40', 'White', 'Female', 34.0382, -118.2889, 20, 510, 101)
(180100513, 2018, 1, 'January', '51-60', 'Black', 'Male', 34.0328, -118.2647, 1, 624, 101)
(180100514, 2018, 1, 'January', '31-40', 'White', 'Female', 34.0428, -118.2532, 1, 230, 108)
(180100516, 2018, 1, 'January', '11-20', 'Other', 'Male', 34.0454, -118.2422, 1, 740, 101)
(180100525, 2018, 1, 'January', '41-50', 'Hispanic', 'Male', 34.0487, -118.2588, 1, 624, 102)
(180100526, 2018, 1, 'January', '21-30', 'Hispanic', 'Female', 34.0487, -118.2588, 1, 624, 102)
(180100529, 2018, 1, 'January', '41-50', 'White', 'Male', 34.0713, -118.2291, 1, 740, 101)


In [3]:
# Query data from crime type table
crime_type_data = conn.execute("SELECT * FROM crime_type")

i = 0
for record in crime_type_data:
    print(record)
    i += 1
    if i == 10:
        break

(121, 'Sexual Assualt or Rape')
(122, 'Sexual Assualt or Rape')
(210, 'Theft, Robbery, or Burglary')
(220, 'Theft, Robbery, or Burglary')
(230, 'Assault')
(231, 'Assault')
(235, 'Child Maltreatment')
(236, 'Domestic Violence')
(237, 'Child Maltreatment')
(310, 'Theft, Robbery, or Burglary')


In [4]:
# Query data from crime table
area_data = conn.execute("SELECT * FROM area")

i = 0
for record in area_data:
    print(record)
    i += 1
    if i == 10:
        break

(1, 'Central')
(2, 'Rampart')
(3, 'Southwest')
(4, 'Hollenbeck')
(5, 'Harbor')
(6, 'Hollywood')
(7, 'Wilshire')
(8, 'West LA')
(9, 'Van Nuys')
(10, 'West Valley')


In [5]:
# Query data from crime table
premise_data = conn.execute("SELECT * FROM premise")

i = 0
for record in premise_data:
    print(record)
    i += 1
    if i == 10:
        break

(101, 'Freeway/Street')
(102, 'Sidewalk')
(103, 'Freeway/Street')
(104, 'Driveway')
(108, 'Parking Lot/Garage')
(109, 'Park')
(110, 'Freeway/Street')
(117, 'Beach')
(119, 'Residence')
(121, 'Yard')


## Joining the Tables

In [6]:
# creating database session
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine=engine, reflect=True)
Base.classes.keys()

['area', 'crime', 'crime_type', 'premise', 'la_crime']

In [7]:
# Collect the names of tables within the database
inspector = inspect(engine)
inspector.get_table_names()

['area', 'crime', 'crime_type', 'la_crime', 'premise']

In [8]:
# Using the inspector to print the column names within the 'crime' table and its types
columns = inspector.get_columns('crime')
for column in columns:
    print(column["name"], column["type"])

incident_id INTEGER
year INTEGER
month INTEGER
month_name VARCHAR
victim_age VARCHAR
victim_ethnicity VARCHAR
victim_gender VARCHAR
lat FLOAT
lon FLOAT
area_code INTEGER
crime_code INTEGER
premise_code INTEGER


In [9]:
# Reflect the database tables
metadata =  MetaData(bind=engine)
metadata.reflect()
crime_table = metadata.tables['crime']
area_table = metadata.tables['area']
crime_type_table = metadata.tables['crime_type']
premise_table = metadata.tables['premise']
crime_table

Table('crime', MetaData(bind=Engine(sqlite:///Resources/la_crime.db)), Column('incident_id', INTEGER(), table=<crime>, primary_key=True, nullable=False), Column('year', INTEGER(), table=<crime>), Column('month', INTEGER(), table=<crime>), Column('month_name', VARCHAR(), table=<crime>), Column('victim_age', VARCHAR(), table=<crime>), Column('victim_ethnicity', VARCHAR(), table=<crime>), Column('victim_gender', VARCHAR(), table=<crime>), Column('lat', FLOAT(), table=<crime>), Column('lon', FLOAT(), table=<crime>), Column('area_code', INTEGER(), ForeignKey('area.area_code'), table=<crime>), Column('crime_code', INTEGER(), ForeignKey('crime_type.crime_code'), table=<crime>), Column('premise_code', INTEGER(), ForeignKey('premise.premise_code'), table=<crime>), schema=None)

In [10]:
# Select the crime table
crime_type_tb = engine.execute(select([crime_type_table]))

# Get all the rows
rows = crime_type_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

49
(121, 'Sexual Assualt or Rape')


In [11]:
# Select the crime type table
crime_tb = engine.execute(select([crime_table]))

# Get all the rows
rows = crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

757085
(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501)


In [12]:
# Select the area table
area_tb = engine.execute(select([area_table]))

# Get all the rows
rows = area_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

21
(1, 'Central')


In [13]:
# Select the premise table
premise = engine.execute(select([premise_table]))

# Get all the rows
rows = premise.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

93
(101, 'Freeway/Street')


In [14]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")

# Execute the SQL query
la_crime = engine.execute(sql)

In [15]:
# Print the column names
print(f"Columns: {la_crime.keys()}")

Columns: ['incident_id', 'year', 'month', 'month_name', 'victim_age', 'victim_ethnicity', 'victim_gender', 'lat', 'lon', 'area_code', 'crime_code', 'premise_code', 'area_code', 'area_name', 'premise_code', 'premise', 'crime_code', 'crime']


In [16]:
# Get all the rows
rows = la_crime.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

757085
(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501, 3, 'Southwest', 501, 'Residence', 624, 'Assault')


In [17]:
# print first 10 rows
i = 0
for row in rows:
    print(row)
    if i == 10:
        break
    i += 1

(10304468, 2020, 1, 'January', '31-40', 'Black', 'Female', 34.0141, -118.2978, 3, 624, 501, 3, 'Southwest', 501, 'Residence', 624, 'Assault')
(20305364, 2019, 1, 'January', '41-50', 'Hispanic', 'Male', 34.0055, -118.2915, 3, 626, 502, 3, 'Southwest', 502, 'Residence', 626, 'Domestic Violence')
(170701073, 2018, 1, 'January', '11-20', 'Black', 'Male', 34.0421, -118.3456, 7, 888, 501, 7, 'Wilshire', 501, 'Residence', 888, 'Trespassing')
(180100001, 2018, 9, 'September', '31-40', 'White', 'Female', 34.0382, -118.2889, 20, 510, 101, 20, 'Olympic', 101, 'Freeway/Street', 510, 'Vehicle Theft')
(180100513, 2018, 1, 'January', '51-60', 'Black', 'Male', 34.0328, -118.2647, 1, 624, 101, 1, 'Central', 101, 'Freeway/Street', 624, 'Assault')
(180100514, 2018, 1, 'January', '31-40', 'White', 'Female', 34.0428, -118.2532, 1, 230, 108, 1, 'Central', 108, 'Parking Lot/Garage', 230, 'Assault')
(180100516, 2018, 1, 'January', '11-20', 'Other', 'Male', 34.0454, -118.2422, 1, 740, 101, 1, 'Central', 101, '

## Creating the final table for running queries 

In [21]:
# creating database session
session = Session(engine)
metadata = MetaData(bind=engine)
Base = automap_base(metadata=metadata)
# Use the Base class to reflect the database tables
Base.prepare(engine = engine, reflect = True)
Base.classes.keys()

['area', 'crime', 'crime_type', 'premise', 'la_crime']

In [ ]:
# Define the la_crime class , which will have a one-to-one relationship with the Area, premise and crime type tables
class CrimeJoined(Base):
    __tablename__ = 'la_crime'
    id = Column(Integer, primary_key=True, autoincrement=True)
    incident_id = Column(Integer)
    year = Column(Integer)
    month = Column(Integer)
    month_name = Column(String)
    victim_age = Column(String)
    victim_ethnicity = Column(String)
    victim_gender = Column(String)
    lat = Column(Float)
    lon = Column(Float)
    area_code = Column(Integer)
    crime_code = Column(Integer)
    premise_code = Column(Integer)
    area_name = Column(String, nullable=False)
    crime = Column(String, nullable=False)
    premise = Column(String, nullable=False)

In [ ]:
# create the table in the database
metadata.create_all(bind=engine, checkfirst=True)

In [ ]:
# Define the SQL query
sql = text("""SELECT * 
            FROM crime
            LEFT JOIN area ON crime.area_code = area.area_code
            LEFT JOIN premise ON crime.premise_code = premise.premise_code
            LEFT JOIN crime_type ON crime.crime_code = crime_type.crime_code""")
# Execute the SQL query
la_crime = engine.execute(sql)
rows = la_crime.fetchall()
print(len(rows))

# loop over the result set and insert each row into the new table
for row in rows:
    print(row)
    new_row = CrimeJoined(incident_id = row.incident_id, 
                       year = row.year,
                       month = row.month,
                       month_name = row.month_name,
                       victim_age = row.victim_age, 
                       victim_ethnicity = row.victim_ethnicity,
                       victim_gender = row. victim_gender,
                       lat = row.lat,
                       lon = row.lon,
                       area_code = row.area_code,
                       crime_code = row.crime_code, 
                       premise_code= row.premise_code,
                       area_name = row.area_name, 
                       crime = row.crime,
                       premise = row.premise)
    session.add(new_row)

# commit the changes
session.commit()

In [18]:
inspector.get_table_names()

['area', 'crime', 'crime_type', 'la_crime', 'premise']

In [ ]:
# Select the la_crime table
la_crime = engine.execute(select([metadata.tables['la_crime_joined']]))

In [ ]:
# Get all the rows
rows = la_crime_tb.fetchall()

# Print the number of rows
print(len(rows))

# Print the first row
print(rows[0])

## Data Queries - using SQLAlchamey (copy to flask app to create functions)

use the table named la_crime for all queries (class is CrimeJoined)

In [22]:
la_crime = engine.execute(select([metadata.tables['la_crime']]))

In [23]:
# Data for overall line chart by year - number of total crimes
sel = [la_crime.year, la_crime.crime]
all_crimes_by_year = session.query(*sel).group_by(la_crime.year).all()
all_crimes_by_year

AttributeError: 'ResultProxy' object has no attribute 'year'

In [ ]:
# Data for overall line chart by year - number of total crimes
sel = [LA_Crime.year, 
       func.avg(Dow.open_price), 
       func.avg(Dow.high_price), 
       func.avg(Dow.low_price), 
       func.avg(Dow.close_price)]
may_averages = session.query(*sel).\
    filter(LA_Crime.year == 2018).\   # in the flask app replace the year 2018 with selected_year var
    filter(LA_Crime.crime == 'Domestic Violence').\   # in the flask app replace 'Domestic Violence' with selected_crime var
    group_by(Dow.stock).\
    order_by(Dow.stock).all()
may_averages


num_rows = session.query(MyTable).count()

'year', 'month', 'month_name', 'victim_age', 'victim_ethnicity', 'victim_gender', 'lat', 'lon', 'area_code', 'crime_code', 'premise_code', 'area_code', 'area_name', 'premise_code', 'premise', 'crime_code', 'crime'